# 관리자 결제 승인 시나리오 (Admin Payment Approval Scenario)

## 시나리오 요약

이 노트북은 다음과 같은 관리자 결제 승인 시나리오를 구현합니다:

1. **결제 요청**: 사용자가 특정 금액과 목적에 대해 결제 승인을 요청
2. **워크플로우 일시정지**: `interrupt`를 통해 관리자의 응답을 기다림
3. **관리자 응답**: 승인 또는 거부 결정
4. **워크플로우 재개**: 관리자의 응답에 따라 프로세스 계속 진행

### 주요 특징:
- **비동기 처리**: 결제 승인 대기 중에도 다른 작업 가능
- **상태 관리**: 각 결제 요청을 별도의 스레드로 관리
- **유연한 응답**: 승인/거부 모두 처리 가능
- **확장성**: 다양한 결제 시나리오에 적용 가능

In [1]:
from dotenv import load_dotenv

# API-KEY 읽어오기
load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model

# 모델 초기화
llm = init_chat_model("openai:gpt-5-nano")
# llm = init_chat_model("google_genai:gemini-2.5-flash")

### 결제 승인 요청 도구 추가

In [4]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.types import Command, interrupt

# 에이전트의 상태 정의 클래스
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 상태 기반 워크플로우 생성
graph_builder = StateGraph(State)

# 결제 승인 요청 도구
@tool
def request_payment_approval(amount: float, description: str, requester: str) -> str:
    """특정 금액과 목적에 대해 관리자에게 결제 승인을 요청합니다."""
    # interrupt: 워크플로우를 일시 중지하고 관리자의 결제 승인을 기다림
    admin_response = interrupt({
        "type": "payment_approval",
        "amount": amount,
        "description": description,
        "requester": requester
    })
    # 관리자의 응답에서 data 반환
    return admin_response["data"]

# 웹 검색 도구
tool = TavilySearch(max_results=2)
# 도구 리스트에 결제 승인 요청 도구 추가
tools = [tool, request_payment_approval]

# LLM이 도구 호출 여부 판단
llm_with_tools = llm.bind_tools(tools)

# chatbot 노드 함수
def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # 병렬 도구 호출 비활성화: 인터럽트 후 툴 중복 호출 방지
    assert len(message.tool_calls) <= 1
    return {"messages": [message]}

# 워크플로우에 chatbot 노드 추가
graph_builder.add_node("chatbot", chatbot)

# tool 노드 워크플로우에 추가
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

# 조건부 라우팅: tools로 이동하거나 END로 이동 (종료)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# tools 노드 실행 후 chatbot 노드로 다시 이동 (도구 결과 처리)
graph_builder.add_edge("tools", "chatbot")

# 워크플로우 시작점에서 chatbot 노드로 이동
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)
graph;

In [5]:
# 그래프를 ASCII 아트로 출력
print(graph.get_graph().draw_ascii())

        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
          +---------+         
          | chatbot |         
          +---------+         
          .         *         
        ..           **       
       .               *      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 


### 결제 승인 요청 시나리오

In [6]:
# 결제 시나리오 실행
user_input = (
    "마케팅 캠페인을 위해 100만원이 필요해. "
    "홍길동 부장의 결제 승인을 요청해줘."
    "다른 추가 메모는 없으니 묻지 말고 그냥 승인 요청해."
)
# 새로운 대화 세션 설정
config = {"configurable": {"thread_id": "payment_approval_1"}}

# 워크플로우를 스트리밍 모드로 실행
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config=config,
    stream_mode="values",
)
# 각 event의 마지막 메시지 출력
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

마케팅 캠페인을 위해 100만원이 필요해. 홍길동 부장의 결제 승인을 요청해줘.다른 추가 메모는 없으니 묻지 말고 그냥 승인 요청해.
================================== Ai Message ==================================
Tool Calls:
  request_payment_approval (call_IM0PSdCSLqVMCTcMoTdLEQVJ)
 Call ID: call_IM0PSdCSLqVMCTcMoTdLEQVJ
  Args:
    amount: 1000000
    description: 마케팅 캠페인 비용 승인 요청
    requester: 홍길동 부장


----------------
```
Command(resume={"data": admin_approval_response})
```
LangGraph에서는 어떤 노드가 외부 입력(예: 관리자 승인, 유저 추가 입력)을 기다리면서 멈출 수 있는데,
이 때 외부에서 응답이 들어오면 Command(resume=...)을 보내서 그래프를 다시 돌립니다.

In [7]:
admin_approval_response = (
    "결제 요청이 승인 되었습니다."
)

# admin_approval_response = (
#     "결제 요청이 거절 되었습니다."
# )

# Command 객체의 resume에 관리자 승인/거부 응답 저장
admin_command = Command(resume={"data": admin_approval_response})

events = graph.stream(
    admin_command,
    config,
    stream_mode="values",
)
# 각 event의 마지막 메시지 출력
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  request_payment_approval (call_IM0PSdCSLqVMCTcMoTdLEQVJ)
 Call ID: call_IM0PSdCSLqVMCTcMoTdLEQVJ
  Args:
    amount: 1000000
    description: 마케팅 캠페인 비용 승인 요청
    requester: 홍길동 부장
================================= Tool Message =================================
Name: request_payment_approval

결제 요청이 승인 되었습니다.
================================== Ai Message ==================================

요청하신 1,000,000원 마케팅 캠페인 비용에 대한 결재가 승인되었습니다. 관련 지출 처리 및 회계 반영을 진행하겠습니다.
